In [ ]:
import os
import csv
import yaml
import wandb
import pickle
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Dense, GRU, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
# from src import read_nz_file, read_jg_file, update_meta_data, split_df, aggregate_files, add_moving_window
from sklearn.model_selection import train_test_split
from wandb.keras import WandbCallback

In [ ]:
def read_preprocessing(folder, file_type: str = 'parquet'):
    if file_type == 'parquet':
        X_train = pd.read_parquet(f'{folder}/X_train.parquet')
        X_test = pd.read_parquet(f'{folder}/X_test.parquet')
        y_train = pd.read_parquet(f'{folder}/y_train.parquet')['y']
        y_test = pd.read_parquet(f'{folder}/y_test.parquet')['y']
    elif file_type == 'pickle':
        with open(f'{folder}/X_train.pickle', 'rb') as f: X_train = pickle.load(f)
        with open(f'{folder}/X_test.pickle', 'rb') as f: X_test = pickle.load(f)
        with open(f'{folder}/y_train.pickle', 'rb') as f: y_train = pickle.load(f)
        with open(f'{folder}/y_test.pickle', 'rb') as f: y_test = pickle.load(f)

    with open(f'{folder}//metadata.yaml') as file:
        settings = yaml.full_load(file)

    return X_train, X_test, y_train, y_test, settings

In [ ]:
# !pip install wandb

## Keras model

In [ ]:
X_train, X_test, y_train, y_test, settings = read_preprocessing(file_type='pickle', folder = './data/sensor/NN_5hz_5sec')

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
settings

In [ ]:
my_train_indexes = [i for i in range(0, y_train.shape[0] - 1)]
random.shuffle(my_train_indexes)

my_test_indexes = [i for i in range(0, y_test.shape[0] - 1)]
random.shuffle(my_test_indexes)

In [ ]:
X_train = X_train.take(my_train_indexes, 0)
y_train = y_train.take(my_train_indexes, 0)
X_test = X_test.take(my_test_indexes, 0)
y_test = y_test.take(my_test_indexes, 0)

In [ ]:
def create_model(first_layer_size = 32, added_dense_layers = 0):
    model = Sequential()
    model.add(GRU(first_layer_size, dropout=0.2, recurrent_dropout=0.2, input_shape=(None, X_train.shape[-1])))
    if added_dense_layers == 1:
        model.add(Dense(128, input_shape = (None, X_train.shape[-1]), activation = 'relu'))
        model.add(Dense(4, activation = 'softmax'))
    elif added_dense_layers == 0:
        model.add(Dense(4, input_shape = (None, X_train.shape[-1]), activation = 'softmax'))

    return model

In [ ]:
wandb.login()

for first_layer_size in [32, 64]:
    for added_dense_layers in [0, 1]:

        model = create_model(first_layer_size = first_layer_size, added_dense_layers = added_dense_layers)

        config={
            "architecture": "RNN",
            "moving_window_size": settings['MOVING_WINDOW_SIZE'],
            "hz": settings['HZ'],
            "step_size": settings['STEP_SIZE'],
            "test_proportion": settings['TEST_PROPORTION'],
            "aggregation": settings['AGGREGATION'],
            "preprocessing": settings['PREPROCESSING'],
            "features": settings['FEATURES'],
            'batch size': 32,
            "epochs": 50,
            "layers": len(model.layers),
            "first_layer_size": first_layer_size,
            "added_dense_layers": added_dense_layers
        }

        run = wandb.init(
            project="CDL1",
            entity="cdl1",
            name=config['architecture'],
            config=config
        )

        # compile model
        model.compile(
            loss = 'categorical_crossentropy',
            optimizer = 'rmsprop',
            metrics = ['accuracy']
        )

        model.fit(
            x = X_train,
            y = y_train,
            epochs = config['epochs'],
            batch_size=config['batch size'],
            validation_data = (X_test, y_test),
            callbacks=[WandbCallback()]
        )

        run.finish()

In [ ]:
predictions = model.predict(X_test)

In [ ]:
pd.Series(predictions.T[0]).apply(lambda x: 1 if x >= 0.5 else 0)